In [2]:
import pyaudio
import wave
import librosa
import numpy as np
from sklearn.cluster import KMeans


In [6]:

def record_audio(duration=5, filename="output.wav"):
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    rate = 44100
    
    p = pyaudio.PyAudio()
    
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk)
    
    print("Recording...")
    frames = []
    
    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    print("Recording complete.")
    
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

def extract_features(filename):
    y, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    features = np.hstack((mfcc_mean, mfcc_std))
    return features

def cluster_features(features, n_clusters=2):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(features.reshape(-1, 1))
    return kmeans.labels_


In [5]:
filename = "output.wav"
#record_audio(filename=filename)
features = extract_features(filename)
labels = cluster_features(features)
print("Cluster labels:", labels)

Cluster labels: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
